## Investigating base substitutions in CRISPR repeats

Ahmed Hasan (ahmed.hasan@mail.utoronto.ca)

Over the course a CRISPR spacer-repeat array, repeats appear to accrue base substitutions. Here, we examine the base spectrum of these substitutions, as well as whether they preferentially occur in certain base positions.

The following is a repeat array found at position 412 in the _Thermotoga maritima_ genome (GenBank accession [NC_000853](https://www.ncbi.nlm.nih.gov/nuccore/NC_000853.1?report=genbank)) using the CRISPRfinder tool.

```
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATAGTTCCTTAGAGGTATGGAAAC
GTTTCAATAGTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATAATTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGATGTACGAAAAA
```

In this demo, these repeats have been saved into a text file called `repeattest.txt`.

In [1]:
from Bio import pairwise2

with open('../../../../repeattest.txt', 'r') as f:
    repeats = [l.rstrip() for l in f.readlines()]

In [2]:
for i in range(len(repeats) - 1):
    repeat1 = repeats[i]
    repeat2 = repeats[i + 1]
    alignments = pairwise2.align.globalxx(repeat1, repeat2)
    for alignment in alignments:
        a = alignment
        print(pairwise2.format_alignment(*alignment))
    break

GTTTCAATACTTCCTTAGAGGTATGGAAAC
||||||||||||||||||||||||||||||
GTTTCAATACTTCCTTAGAGGTATGGAAAC
  Score=30



In [3]:
# what do those values mean? score - mismatches - matches?
alignments = pairwise2.align.globalxx(repeats[38], repeats[39])
for alignment in alignments:
    print(alignment, '\n')
    print(pairwise2.format_alignment(*alignment))
    


('GTTTCAATAA-TTCCTTAGAGGTATGGAAAC', 'GTTTCAAT-ACTTCCTTAGAGGTATGGAAAC', 29.0, 0, 31) 

GTTTCAATAA-TTCCTTAGAGGTATGGAAAC
|||||||||||||||||||||||||||||||
GTTTCAAT-ACTTCCTTAGAGGTATGGAAAC
  Score=29

('GTTTCAATAA-TTCCTTAGAGGTATGGAAAC', 'GTTTCAATA-CTTCCTTAGAGGTATGGAAAC', 29.0, 0, 31) 

GTTTCAATAA-TTCCTTAGAGGTATGGAAAC
|||||||||||||||||||||||||||||||
GTTTCAATA-CTTCCTTAGAGGTATGGAAAC
  Score=29

('GTTTCAATAATTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGAGGTATGGAAAC', 29.0, 0, 30) 

GTTTCAATAATTCCTTAGAGGTATGGAAAC
||||||||||||||||||||||||||||||
GTTTCAATACTTCCTTAGAGGTATGGAAAC
  Score=29



From the [source code](http://biopython.org/DIST/docs/api/Bio.pairwise2-pysrc.html):

>alignments is a list of tuples (seqA, seqB, score, begin, end). 
seqA and seqB are strings showing the alignment between the 
sequences.  score is the score of the alignment.  begin and end 
are indexes into seqA and seqB that indicate the where the 
alignment occurs

We should try incorporating BLASTN's [default gap opening/extending penalities](https://www.arabidopsis.org/Blast/BLASToptions.jsp):

|type|penalty|
|---|---|
|gap opening|5|
|gap extending|2|
|mismatch|3|

In [4]:
alignments = pairwise2.align.globalms(repeats[38], repeats[39], 1, -3, -5, -2) 
# 1 for match, -3 for mismatch, -5 for gap open, -2 for gap extend
for alignment in alignments:
    print(alignment, '\n')
    print(pairwise2.format_alignment(*alignment))

('GTTTCAATAATTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGAGGTATGGAAAC', 26.0, 0, 30) 

GTTTCAATAATTCCTTAGAGGTATGGAAAC
||||||||||||||||||||||||||||||
GTTTCAATACTTCCTTAGAGGTATGGAAAC
  Score=26



In [5]:
alignments = pairwise2.align.globalms(repeats[38] + repeats[38], repeats[39] + repeats[39], 1, -3, -5, -2) 
# 1 for match, -3 for mismatch, -5 for gap open, -2 for gap extend
for alignment in alignments:
    print(alignment, '\n')
    print(pairwise2.format_alignment(*alignment))

('GTTTCAATAATTCCTTAGAGGTATGGAAACGTTTCAATAATTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGAGGTATGGAAACGTTTCAATACTTCCTTAGAGGTATGGAAAC', 52.0, 0, 60) 

GTTTCAATAATTCCTTAGAGGTATGGAAACGTTTCAATAATTCCTTAGAGGTATGGAAAC
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
GTTTCAATACTTCCTTAGAGGTATGGAAACGTTTCAATACTTCCTTAGAGGTATGGAAAC
  Score=52



Seems these are pretty stringent gap open/extend penalties and, at least in this example array, suppress `pairwise2` from returning any gap-containing alignments at all.

It also seems scores can be measured using

$$\frac{score}{end - begin}$$

since this maxes out at 1 when we have a perfect match.

Now that we're effectively suppressing gaps, we also need to consider the _positions_ at which mismatches are occuring, in addition to the nucleotide changes that are taking place. These could potentially be stored in some sort of dictionary format?

In [6]:
# base substitution type
alignments = pairwise2.align.globalms(repeats[38], repeats[39], 1, -3, -5, -2) 
# 1 for match, -3 for mismatch, -5 for gap open, -2 for gap extend
for alignment in alignments:
    print(alignment, '\n')
    print(pairwise2.format_alignment(*alignment))
    seq1, seq2, score, start, end = alignment # unpack
    total = end - start
    ratio = score / total
    print(ratio)
    changes = {}
    assert len(seq1) == len(seq2)
    for i in range(len(seq1)):
        prev = seq1[i]
        new = seq2[i]
        if prev != new:
            dinucleotide = prev + new
            if dinucleotide not in changes.keys():
                changes[prev + new] = 1
            elif dinucleotide in changes.keys():
                changes[prev + new] += 1
    print(changes)
    
# now make a mismatches dict? or combine with changes??

('GTTTCAATAATTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGAGGTATGGAAAC', 26.0, 0, 30) 

GTTTCAATAATTCCTTAGAGGTATGGAAAC
||||||||||||||||||||||||||||||
GTTTCAATACTTCCTTAGAGGTATGGAAAC
  Score=26

0.8666666666666667
{'AC': 1}


Now for the main class:

In [7]:
import pandas as pd

class repeatarray(object):
    
    def __init__(self, repeatlist):
        self.all = repeatlist
        self.initial = repeatlist[0]
        self.final = repeatlist[-1]
        self.num_repeats = len(repeatlist)
        assert len(set([len(repeat) for repeat in repeatlist])) == 1
    
    def align2(self, index1, index2, match = 1, mismatch = -3, gap_open = -5, gap_extend = -2):
        '''Given the indices of two repeats in the repeat array, returns alignment'''
        alignments = pairwise2.align.globalms(self.all[index1], self.all[index2], 
                                              match, mismatch, gap_open, gap_extend)
        return alignments[0] # best alignment
    
    def all_scores(self, match = 1, mismatch = -3, gap_open = -5, gap_extend = -2, df = False):
        '''For each repeat, returns alignment score between repeat + next repeat in array'''
        scores = {}
        for i in range(len(self.all) - 1):
            alignment = self.align2(i, i + 1, match, mismatch, gap_open, gap_extend)
            seq1, seq2, score, start, end = alignment
            scores[i] = score / (end - start)
        
        if df:
            return pd.Series(scores).to_frame().reset_index()
        else:
            return scores
    
    def substitution_counts(self, match = 1, mismatch = -3, gap_open = -5, gap_extend = -2, df = False):
        '''Returns count of base substitutions. Substitutions are represented by strings, where
        "AT" means A -> T.'''
        mismatches = {}
        for i in range(self.num_repeats - 1):
            seq1 = self.all[i]
            seq2 = self.all[i + 1]
            
            alignment = self.align2(i, i + 1, match, mismatch, gap_open, gap_extend)
            current_mismatches = [[i, seq1[i], seq2[i]] for i in range(len(seq1)) if seq1[i] != seq2[i]]
            dinucleotides = [m[1] + m[2] for m in current_mismatches]
            
            for d in dinucleotides:
                if d in mismatches.keys():
                    mismatches[d] += 1
                elif d not in mismatches.keys():
                    mismatches[d] = 1   
        if df:
            return pd.Series(mismatches).to_frame().reset_index()
        else:
            return mismatches
    
    def mismatch_positions(self, match = 1, mismatch = -3, gap_open = -5, gap_extend = -2, df = False):
        '''Returns count of mismatches at all base pair positions across repeat array.'''
        mismatches = dict.fromkeys(list(range(len(self.initial))), 0)
        for i in range(self.num_repeats - 1):
            seq1 = self.all[i]
            seq2 = self.all[i + 1]
            
            alignment = self.align2(i, i + 1, match, mismatch, gap_open, gap_extend)
            mismatch_positions = [i for i in range(len(seq1)) if seq1[i] != seq2[i]]
            for m in mismatch_positions:
                mismatches[m] += 1
                
        if df:
            return pd.Series(mismatches).to_frame().reset_index()
        else:
            return mismatches
            

In [8]:
# some convenience methods
array = repeatarray(repeats)
print(array.initial)
print(array.final)
print(array.all[:2]) # entire array
print(array.num_repeats)
print(array.align2(0, -1)) # align first and last repeat - return as pairwise2 object

GTTTCAATACTTCCTTAGAGGTATGGAAAC
GTTTCAATACTTCCTTAGATGTACGAAAAA
['GTTTCAATACTTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGAGGTATGGAAAC']
41
('GTTTCAATACTTCCTTAGAGGTATGGAAAC', 'GTTTCAATACTTCCTTAGATGTACGAAAAA', 14.0, 0, 30)


In [9]:
# array-wide method - return all consecutive alignments 
# in the form (score / max possible value of score)
array.all_scores()

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 1.0,
 12: 1.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 1.0,
 20: 0.8666666666666667,
 21: 1.0,
 22: 1.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 0.8666666666666667,
 35: 1.0,
 36: 0.8666666666666667,
 37: 0.8666666666666667,
 38: 0.8666666666666667,
 39: 0.4666666666666667}

In [10]:
# can also be returned as a pandas df
array.all_scores(df = True)

,index,0
0,0,1.000000
1,1,1.000000
2,2,1.000000
3,3,1.000000
4,4,1.000000
5,5,1.000000
6,6,1.000000
7,7,1.000000
8,8,1.000000
9,9,1.000000


In [11]:
# which positions have the most mismatches?
array.mismatch_positions(df = True)

,index,0
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,5


In [12]:
# what are the most common base substitutions?
array.substitution_counts(df = True)

,index,0
0,AC,1
1,AG,1
2,CA,3
3,GA,1
4,GC,1
5,GT,1
6,TC,1
